In [1]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit,UN
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRGenerator
from mindquantum.core.circuit import change_param_name
from mindspore.nn import Adam  
import random
from mindspore import Tensor,ops,Parameter
from mindquantum.core.gates import UnivMathGate
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer,MQOps
from torchvision import datasets
import mindspore.numpy as mnp
import sys
from typing import Union
sys.path.append('..')
from Test_tool import Test_ansatz
from data_processing import X_train,X_test,y_train,y_test
from mindspore.train import Accuracy, Model, LossMonitor  
from mindspore.dataset import NumpySlicesDataset


/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [2]:
Pauli_ops=\
['X0 X1','X1 X2','X3 X4','X4 X5','X6 X7','X7 X0',
 'Y0 Y1','Y1 Y2','Y3 Y4','Y4 Y5','Y6 Y7','Y7 Y0',
 'Z0 Z1','Z1 Z2','Z3 Z4','Z4 Z5','Z6 Z7','Z7 Z0',\
 'X0 Y1 Z2 X3','Y1 Z2 X3 X4']

unparameterized_circuit = \
[UN(Z, maps_obj = [0,1,2,3], maps_ctrl = [4,5,6,7]),
 UN(Z, maps_obj = [4,5,6,7], maps_ctrl = [0,1,2,3]),
 UN(X, maps_obj = [0,1,2,3], maps_ctrl = [4,5,6,7]),
 UN(X, maps_obj = [4,5,6,7], maps_ctrl = [0,1,2,3]),
 UN(Y, maps_obj = [0,1,2,3], maps_ctrl = [4,5,6,7]),
 UN(Y, maps_obj = [4,5,6,7], maps_ctrl = [0,1,2,3]),
 ]
ansatz_pr = PRGenerator('ansatz')
parameterized_circuit=[TimeEvolution(QubitOperator(i,ansatz_pr.new()),time=0.5).circuit for i in Pauli_ops]

In [3]:
shape_parametized = len(parameterized_circuit)
shape_unparameterized = len(unparameterized_circuit)
num_layer=12
shape_nnp = (num_layer,shape_parametized)
shape_stp = (num_layer,shape_unparameterized+shape_parametized)
stddev = 0.03
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)
ops_onehot = ops.OneHot(axis=-1)

In [4]:
from DQAS_tool import best_from_structure

In [11]:
can = best_from_structure(structures=stp)
standard_stp = ops_onehot(best_from_structure(structures=stp),
                          shape_unparameterized+shape_parametized,
                          ms.Tensor(1,ms.int32),ms.Tensor(0,ms.int32))

standard_stp

Tensor(shape=[12, 26], dtype=Int32, value=
[[0, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 1, 0, 0],
 ...
 [0, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0],
 [0, 0, 0 ... 0, 0, 0]])

In [12]:
can

Tensor(shape=[12], dtype=Int32, value= [11, 20, 23, 19,  3, 15, 14, 19,  3, 11, 22,  4])

In [34]:
def Mindspore_ansatz3(Structure_p:np.array,
                     parameterized_pool:list[Circuit],
                     unparameterized_pool:list[Circuit],
                     num_layer:int=6,
                     n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    更新了非参数化门的算符池引入
    Structure_p:np.array DQAS中的权重参数,
    Ansatz_p:np.array  DQAS中的Ansatz参数,
    
    """
    if Structure_p.shape[0] != num_layer:
        raise ValueError('Structure_p shape must be equal to num_layer')
    
    if Structure_p.shape[1] != len(parameterized_pool)+len(unparameterized_pool):
        raise ValueError('Structure_p shape must be equal to size of pool')
    # softmax = ops.Softmax()
    # my_stp = softmax(Tensor(Structure_p, ms.float32))
    if isinstance(Structure_p, np.ndarray):
        my_stp = ms.Tensor(Structure_p, ms.float32)
    else:
        my_stp = Structure_p
        
    prg = PRGenerator('encoder')
    nqbits = n_qbits
    encoder = Circuit()
    # encoder += UN(H, nqbits)                                 
    for i in range(nqbits):                                  
        encoder += RY(prg.new()).on(i)    
    encoder = encoder.as_encoder()             
        
    ansatz = Circuit()
    pr_gen = PRGenerator('ansatz')
    #print(my_stp.shape)
    for layer_index in range(my_stp.shape[0]):
        for op_index in range(my_stp.shape[1]):
            if my_stp[layer_index,op_index] == 0:
                continue
            if op_index < len(parameterized_pool):
                before_ansatz = parameterized_pool[op_index]
                before_ansatz = change_param_name(circuit_fn=before_ansatz,name_map={before_ansatz.ansatz_params_name[0]:f'ansatz{layer_index}'})
                ansatz += before_ansatz
            else:
                ansatz += unparameterized_pool[op_index-len(parameterized_pool)]
    
    
    
    finnal_ansatz = encoder.as_encoder() + ansatz.as_ansatz()
    # print(finnal_ansatz)
    # name_map = dict(zip(finnal_ansatz.ansatz_params_name,[f'ansatz{i}'for i in range(len(finnal_ansatz.ansatz_params_name))]))
    # finnal_ansatz = change_param_name(circuit_fn=finnal_ansatz,name_map=name_map)
    return finnal_ansatz



In [35]:
ansatz = Mindspore_ansatz3(Structure_p=standard_stp,parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,num_layer=12,n_qbits=8)

In [37]:
ansatz.ansatz_params_name

['ansatz0',
 'ansatz3',
 'ansatz4',
 'ansatz5',
 'ansatz6',
 'ansatz7',
 'ansatz8',
 'ansatz9',
 'ansatz11']

In [38]:
ansatz.encoder_params_name

['encoder0',
 'encoder1',
 'encoder2',
 'encoder3',
 'encoder4',
 'encoder5',
 'encoder6',
 'encoder7']

In [47]:
acc = Test_ansatz(ansatz=ansatz,learning_rate=0.3,epochs=20)

epoch: 1 step: 20, loss is 0.7484782338142395
epoch: 1 step: 40, loss is 0.7556942701339722
epoch: 2 step: 11, loss is 0.6591635942459106
epoch: 2 step: 31, loss is 0.6708909273147583
epoch: 3 step: 2, loss is 0.6348642110824585
epoch: 3 step: 22, loss is 0.669664740562439
epoch: 3 step: 42, loss is 0.7045347690582275
epoch: 4 step: 13, loss is 0.6332474946975708
epoch: 4 step: 33, loss is 0.7044292688369751
epoch: 5 step: 4, loss is 0.5578922033309937
epoch: 5 step: 24, loss is 0.682957112789154
epoch: 5 step: 44, loss is 0.6039729118347168
epoch: 6 step: 15, loss is 0.7316300272941589
epoch: 6 step: 35, loss is 0.6445085406303406
epoch: 7 step: 6, loss is 0.6514734029769897
epoch: 7 step: 26, loss is 0.6697326302528381
epoch: 7 step: 46, loss is 0.6712834239006042
epoch: 8 step: 17, loss is 0.5959731936454773
epoch: 8 step: 37, loss is 0.5690423250198364
epoch: 9 step: 8, loss is 0.7655727863311768
epoch: 9 step: 28, loss is 0.6961480975151062
epoch: 9 step: 48, loss is 0.60327571630